In [ ]:
# Premonitor: main.py
# This is the main entry point for the Premonitor application.
# It runs a continuous loop to monitor the environment, processes data with
# AI models, applies fusion logic, and triggers non-blocking alerts.
# This script is intended to be run on the Raspberry Pi.

import time
import numpy as np
import tensorflow as tf
from datetime import datetime
import os
import json

# Import our custom project files
import config
import alert_manager
# For the MVP, we use the mock hardware. To switch to real hardware,
# you would change this line to: import hardware_drivers as hardware
import mock_hardware as hardware

# --- Global variables for loaded AI models ---
thermal_interpreter = None
acoustic_interpreter = None
# Add other models like LSTM AE here in the future.

def load_models():
    """
    Loads the trained and quantized .tflite models into memory using the
    TensorFlow Lite interpreter for high efficiency on the Pi.
    """
    global thermal_interpreter, acoustic_interpreter
    print("MAIN: Loading AI models...")
    try:
        thermal_interpreter = tf.lite.Interpreter(model_path=config.THERMAL_MODEL_PATH)
        thermal_interpreter.allocate_tensors()
        print(f"MAIN: Successfully loaded thermal model from {config.THERMAL_MODEL_PATH}")

        acoustic_interpreter = tf.lite.Interpreter(model_path=config.ACOUSTIC_MODEL_PATH)
        acoustic_interpreter.allocate_tensors()
        print(f"MAIN: Successfully loaded acoustic model from {config.ACOUSTIC_MODEL_PATH}")

    except Exception as e:
        print(f"MAIN: CRITICAL ERROR - Failed to load one or more models: {e}")
        print("MAIN: The application cannot run without the AI models. Exiting.")
        exit()

def run_inference(interpreter, input_data):
    """
    A generic function to run inference on a loaded TFLite interpreter.
    """
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Prepare the input tensor (needs to be in a batch of 1)
    input_tensor = np.expand_dims(input_data, axis=0).astype(np.float32)

    # Check if input shape matches model's expected shape
    if not np.array_equal(input_details[0]['shape'][1:], input_data.shape):
         print(f"Error: Input data shape {input_data.shape} does not match model expected shape {input_details[0]['shape'][1:]}")
         return 0.0 # Return a non-anomaly score

    interpreter.set_tensor(input_details[0]['index'], input_tensor)
    interpreter.invoke()
    prediction = interpreter.get_tensor(output_details[0]['index'])[0]

    # Assuming the model's output is a single value from a sigmoid function
    return prediction[0]

def main():
    """
    The main function that orchestrates the Premonitor system.
    """
    print("--- Starting Premonitor System ---")

    # --- Initialization ---
    load_models()
    hardware.initialize_mock_data() # Initialize our virtual sensors

    # TODO: In a V2.0 product, load device-specific config from a JSON file.
    # For now, we use the defaults from config.py.

    # --- Main Monitoring Loop ---
    print("\n--- Starting main monitoring loop ---")
    while True:
        try:
            current_time = datetime.now().strftime('%H:%M:%S')
            print(f"\n--- New check cycle at {current_time} ---")

            # 1. Read from sensors (using mock hardware for now)
            thermal_image = hardware.read_thermal_image()
            acoustic_spectrogram = hardware.read_acoustic_spectrogram()
            gas_reading = hardware.read_gas_sensor()

            # 2. Process data with AI models to get confidence scores
            thermal_score = run_inference(thermal_interpreter, thermal_image)
            acoustic_score = run_inference(acoustic_interpreter, acoustic_spectrogram)

            if config.DEBUG_MODE:
                print(f"MAIN: Thermal Score={thermal_score:.2f}, Acoustic Score={acoustic_score:.2f}, Gas Reading={gas_reading}")

            # 3. Apply Sensor Fusion & Alerting Logic
            # This is the core intelligence of the system.

            # Case 1: High-confidence single-sensor alert (critical)
            if thermal_score > config.THERMAL_ANOMALY_CONFIDENCE:
                details = f"A critical thermal anomaly was detected with a high confidence score of {thermal_score:.2f}."
                alert_manager.send_alert_in_background("CRITICAL: High-Confidence Thermal Anomaly", details)

            elif acoustic_score > config.ACOUSTIC_ANOMALY_CONFIDENCE:
                details = f"A critical acoustic anomaly was detected with a high confidence score of {acoustic_score:.2f}."
                alert_manager.send_alert_in_background("CRITICAL: High-Confidence Acoustic Anomaly", details)

            # Case 2: Correlated multi-sensor alert (proactive)
            elif thermal_score > config.FUSION_CORRELATION_CONFIDENCE and acoustic_score > config.FUSION_CORRELATION_CONFIDENCE:
                details = (f"A potential issue was detected by correlating multiple weak signals.\n"
                           f"- Thermal anomaly score: {thermal_score:.2f} (Threshold: {config.FUSION_CORRELATION_CONFIDENCE})\n"
                           f"- Acoustic anomaly score: {acoustic_score:.2f} (Threshold: {config.FUSION_CORRELATION_CONFIDENCE})")
                alert_manager.send_alert_in_background("WARNING: Correlated Anomaly Detected", details)

            # Case 3: Simple threshold alert for basic sensors
            if gas_reading > config.GAS_ANALOG_THRESHOLD:
                details = f"The gas sensor reading was {gas_reading}, which is above the threshold of {config.GAS_ANALOG_THRESHOLD}."
                alert_manager.send_alert_in_background("WARNING: High Gas Level Detected", details)

            # 4. Wait for the next cycle
            time.sleep(config.SENSOR_READ_INTERVAL)

        except KeyboardInterrupt:
            print("\n--- Shutdown signal received. Exiting Premonitor. ---")
            break
        except Exception as e:
            print(f"MAIN: An unhandled error occurred in the main loop: {e}")
            # In a real product, you might want to send a "System Health" alert here.
            time.sleep(30) # Wait a bit before retrying to avoid spamming errors

if __name__ == "__main__":
    main()